In [54]:
import pyro
import pyro.distributions as dist
import torch
from pyro import poutine
from pyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO, config_enumerate, infer_discrete
from pyro.infer.autoguide import AutoNormal
from pyro.optim import Adam
from torch.distributions import constraints
from pyro import poutine
from pyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO, config_enumerate, infer_discrete

In [60]:
def model(data):
    z = pyro.sample("z", dist.Categorical(torch.ones(2)))
    if z == torch.tensor(0):
        for i in range(len(data)):
            pyro.sample("obs_{}".format(i), dist.Categorical(torch.ones(2)), obs=data[i])
    else:
        for i in range(len(data)):
            pyro.sample("obs_{}".format(i), dist.Categorical(torch.ones(10)), obs=data[i])

def guide(data):
    p = pyro.param("p", torch.rand(2), constraint=constraints.simplex)
    pyro.sample("z", dist.Categorical(p))

In [62]:
data = [0,0,0,1,1,1,1,0]

data = [torch.tensor(i) for i in data]

adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

n_steps = 1000
# do gradient steps
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')

.

ValueError: Error while computing log_prob at site 'z':
Expected value argument (Tensor of shape ()) to be within the support (IntegerInterval(lower_bound=0, upper_bound=1)) of the distribution Categorical(probs: torch.Size([2])), but found invalid values:
2
Trace Shapes:  
 Param Sites:  
Sample Sites:  
       z dist |
        value |

In [48]:
a = pyro.get_param_store()
a.get_all_param_names()

dict_keys([])